In [1]:
import pandas as pd
import workdays
import datetime
from datetime import datetime, timedelta, date

from xbbg import blp
import pdblp
con = pdblp.BCon(timeout=10000)
con.start()
# https://data.bloomberglp.com/labs/sites/2/2013/12/blpapi-developers-guide-1.38.pdf
# https://data.bloomberglp.com/professional/sites/10/2017/03/BLPAPI-Core-Developer-Guide.pdf

In [2]:
def BDP(L,YK):
    T = []
    T1 = []
    for i in range(len(L)):
        T += [L[i] + "A " + YK[i] ]
        T1 += [L[i] + "1 " + YK[i] ]

    BDP = blp.bdp(tickers=T, flds=["name","currency","country_iso","exch_code","undl_spot_ticker"]).loc[T,:]
    tmp = BDP["undl_spot_ticker"] + " Index"
    tmp = blp.bdp(tickers=tmp, flds="country_iso").loc[tmp,:].iloc[:,0]
    BDP["risk"] = list(tmp)

    BDP.insert(0,"ticker", L)
    BDP = BDP.reset_index(drop=True)

    BDP1 = blp.bdp(tickers=T1, flds=["contract_value","volume_avg_20d","volatility_20d"]).loc[T1,:]
    BDP1.columns = ["contract_value","volume","vola"]
    BDP = pd.concat([BDP, BDP1.reset_index(drop=True)], axis=1)

    BDP = BDP.rename(columns={"currency":"crncy","country_iso":"exch","exch_code":"exch_code","undl_spot_ticker":"undl"})
    
    tmp = ["contract_value","volume"]
    for j in tmp:
        BDP[j] = BDP[j].apply("{:,.0f}".format)
        
    for j in ["vola"]:
        BDP[j] = round(BDP[j],2).values
        
    tmp = []
    for i in range(len(L)):
        tmp  += [(con.bdh(T1[i], "px_last", "19991201", "20220101", elms = [("periodicitySelection","MONTHLY")]).reset_index().iloc[0,0]).strftime("%Y%m%d")]
    BDP["initinal_t/d"] = tmp
    BDP = BDP.replace("19991231", "-")
    
    return BDP
    
L= ["ES","NQ","RTY","VG","GX","Z ","PT","XP","TP","HI","XU","IH","TWT"]
# L= ["ES","NQ"]
YK = ["Index"] * len(L)

print("Equity Futures")
BDP = BDP(L,YK)
print(BDP.shape)
BDP

Equity Futures
(13, 11)


,ticker,name,crncy,exch,exch_code,undl,risk,contract_value,volume,vola,initinal_t/d
0,ES,S&P500 EMINI FUT Sep23,USD,US,CME,SPX,US,"218,075","1,725,170",11.91,-
1,NQ,NASDAQ 100 E-MINI Sep23,USD,US,CME,NDX,US,"296,235","657,500",18.07,-
2,RTY,E-Mini Russ 2000 Sep23,USD,US,CME,RTY,US,"94,500","231,412",23.84,20170731
3,VG,EURO STOXX 50 Jun23,EUR,DE,EUX,SX5E,EU,"43,230","769,027",16.05,-
4,GX,DAX INDEX FUTURE Jun23,EUR,DE,EUX,DAX,DE,"402,250","57,482",15.22,-
5,Z,FTSE 100 IDX FUT Jun23,GBP,GB,ICF,UKX,GB,"75,815","103,940",12.66,-
6,PT,S&P/TSX 60 IX FUT Jun23,CAD,CA,MSE,SPTSX60,CA,"238,780","25,905",13.10,-
7,XP,SPI 200 FUTURES Jun23,AUD,AU,SFE,AS51,AU,"177,550","48,511",11.47,20000531
8,TP,TOPIX INDX FUTR Sep23,JPY,JP,OSE,TPX,JP,"22,435,000","103,346",15.05,-
9,HI,HANG SENG IDX FUT Jun23,HKD,HK,HKG,HSI,HK,"968,650","124,496",24.25,-


In [3]:
def BDP(L,YK):
    T = []
    T1 = []
    for i in range(len(L)):
        T += [L[i] + "A " + YK[i] ]
        T1 += [L[i] + "1 " + YK[i] ]

    BDP = blp.bdp(tickers=T, flds=["name","currency","country_iso","exch_code","fut_ctd_isin"]).loc[T,:]

    isin = BDP.loc[T,["fut_ctd_isin"]].dropna(how='all')
    isin = [i for i in isin["fut_ctd_isin"]]

    tmp0 = ["year","dur","yld"]
    tmp1 = ["mty_years","dur_adj_bid","yld_ytm_bid"]


    BDP1 = blp.bdp(tickers = [i+" Corp" for i in isin], flds=tmp1)
    
    for i in isin:
        for j in range(len(tmp0)):
            BDP.loc[BDP["fut_ctd_isin"]==i,tmp0[j]] = BDP1.loc[i+" Corp",tmp1[j]]

    BDP.insert(0,"ticker", L)
    BDP = BDP.reset_index(drop=True)

    BDP1 = blp.bdp(tickers=T1, flds=["contract_value","volume_avg_20d","volatility_20d"]).loc[T1,:]
    BDP1.columns = ["contract_value","volume","vola"]
    BDP = pd.concat([BDP, BDP1.reset_index(drop=True)], axis=1)
    

    BDP = BDP.rename(columns={"currency":"crncy","country_iso":"exch","exch_code":"exch_code","undl_spot_ticker":"undl"})
    
    for j in ["contract_value","volume"]:
        BDP[j] = BDP[j].apply("{:,.0f}".format)
        
    for j in ["year","dur","yld","vola"]:
        BDP[j] = round(BDP[j],2).values
        
    tmp = []
    for i in range(len(L)):
        tmp  += [(con.bdh(T1[i], "px_last", "19991201", "20210101", elms = [("periodicitySelection","MONTHLY")]).reset_index().iloc[0,0]).strftime("%Y%m%d")]
    BDP["initinal_t/d"] = tmp
    BDP = BDP.replace("19991231", "-")
    
    return BDP
    
L = ["TU","FV","TY","US","WN","DU","OE","RX","UB","OAT","IK","G ","CN","XM","JB"]
# L = ["TU","FV"]
YK = ["Comdty"] * len(L)

print("Bond Futures")
BDP = BDP(L,YK)
print(BDP.shape)
BDP

Bond Futures
(15, 13)


,ticker,name,crncy,exch,exch_code,fut_ctd_isin,year,dur,yld,contract_value,volume,vola,initinal_t/d
0,TU,US 2YR NOTE (CBT) Sep23,USD,US,CBT,US91282CEU18,2.01,1.89,4.59,"204,969","829,407",2.24,-
1,FV,US 5YR NOTE (CBT) Sep23,USD,US,CBT,US91282CFZ95,4.47,4.06,3.97,"108,352","1,488,080",4.84,-
2,TY,US 10YR NOTE (CBT)Sep23,USD,US,CBT,US91282CGS44,6.80,5.92,3.82,"113,578","1,872,836",6.32,-
3,US,US LONG BOND(CBT) Sep23,USD,US,CBT,US912810QC53,16.17,11.48,3.82,"127,906","369,230",9.72,-
4,WN,US ULTRA BOND CBT Sep23,USD,US,CBT,US912810SE91,25.43,16.40,3.99,"136,562","291,156",10.72,20100129
5,DU,EURO-SCHATZ FUT Sep23,EUR,DE,EUX,DE000BU22015,2.00,1.91,2.87,"105,520","531,559",1.63,-
6,OE,EURO-BOBL FUTURE Sep23,EUR,DE,EUX,DE000BU25000,4.83,4.49,2.37,"116,920","620,654",4.54,-
7,RX,EURO-BUND FUTURE Sep23,EUR,DE,EUX,DE0001102606,9.17,8.22,2.30,"134,630","745,542",8.01,-
8,UB,EURO-BUXL 30Y BND Sep23,EUR,DE,EUX,DE0001102432,25.17,20.41,2.43,"137,080","87,479",18.55,-
9,OAT,Euro-OAT Future Sep23,EUR,DE,EUX,FR001400H7V7,9.95,8.50,2.86,"129,200","202,370",8.36,20120430


In [4]:
def BDP(L,YK):
    T = []
    T1 = []
    for i in range(len(L)):
        T += [L[i] + "A " + YK[i] ]
        T1 += [L[i] + "1 " + YK[i] ]

    BDP = blp.bdp(tickers=T, flds=["name","currency","country_iso","exch_code","undl_spot_ticker"]).loc[T,:]

    BDP.insert(0,"ticker", L)
    BDP = BDP.reset_index(drop=True)

    BDP1 = blp.bdp(tickers=T1, flds=["contract_value","volume_avg_20d","volatility_20d"]).loc[T1,:]
    BDP1.columns = ["contract_value","volume","vola"]
    BDP = pd.concat([BDP, BDP1.reset_index(drop=True)], axis=1)

    BDP = BDP.rename(columns={"currency":"crncy","country_iso":"exch","exch_code":"exch_code","undl_spot_ticker":"undl"})
    
    tmp = ["contract_value","volume"]
    for j in tmp:
        BDP[j] = BDP[j].apply("{:,.0f}".format)
        
    for j in ["vola"]:
        BDP[j] = round(BDP[j],2).values
        
    tmp = []
    for i in range(len(L)):
        tmp  += [(con.bdh(T1[i], "px_last", "19991201", "20220101", elms = [("periodicitySelection","MONTHLY")]).reset_index().iloc[0,0]).strftime("%Y%m%d")]
    BDP["initinal_t/d"] = tmp
    BDP = BDP.replace("19991231", "-")
    
    return BDP
    
L= ["CL","CO","GC"]
YK = ["Comdty"] * len(L)

print("Commodity Futures")
BDP = BDP(L,YK)
print(BDP.shape)
BDP

Commodity Futures
(3, 10)


,ticker,name,crncy,exch,exch_code,undl,contract_value,volume,vola,initinal_t/d
0,CL,WTI CRUDE FUTURE Jul23,USD,US,NYM,USCRWTIC,"68,040","299,305",23.97,-
1,CO,BRENT CRUDE FUTR Aug23,USD,GB,ICE,COY,"72,820","244,271",31.87,-
2,GC,GOLD 100 OZ FUTR Aug23,USD,US,CMX,NaN,"197,760","131,477",12.86,-


In [5]:
def BDP(L):
    T = ["CSBS" + i + " TMUQ Curncy" for i in L]  #Mitsubishi TTM
    tmp = ["px_last","volatility_60d","rsi_30d"]
    BDP = blp.bdp(tickers=T, flds=tmp).loc[T,:]
    BDP = BDP.loc[T,:]

    for i in tmp:
        BDP[i.lower()] = round(BDP[i.lower()].astype(float),2)
    BDP.insert(0, "currency", L )
    BDP.insert(1, "ticker", T)
    BDP = BDP.reset_index(drop=True)
    BDP = BDP.rename(columns={"volatility_60d":"vola"})

    return BDP

L = ["USD","EUR","GBP","CAD","AUD","NZD","CHF","NOK","SEK","HKD","CNY"]

print("FX")
BDP = BDP(L)
print(BDP.shape)
BDP

FX
(11, 5)


,currency,ticker,px_last,vola,rsi_30d
0,USD,CSBSUSD TMUQ Curncy,139.43,10.14,56.94
1,EUR,CSBSEUR TMUQ Curncy,149.82,10.88,55.60
2,GBP,CSBSGBP TMUQ Curncy,175.32,10.12,63.77
3,CAD,CSBSCAD TMUQ Curncy,104.49,10.70,60.36
4,AUD,CSBSAUD TMUQ Curncy,94.09,12.24,59.28
5,NZD,CSBSNZD TMUQ Curncy,85.37,12.84,52.94
6,CHF,CSBSCHF TMUQ Curncy,154.27,10.65,59.27
7,NOK,CSBSNOK TMUQ Curncy,12.94,13.90,53.48
8,SEK,CSBSSEK TMUQ Curncy,12.86,12.07,47.46
9,HKD,CSBSHKD TMUQ Curncy,17.79,10.12,57.22


In [7]:
def BDP(L):
    T = [i + " Index" for i in L]   
    BDP = blp.bdp(tickers=T, flds=["Security_name","long_comp_name","Currency","volatility_260d"]).loc[T,:]
    BDP.insert(0, "ticker", T)
    BDP = BDP.reset_index(drop=True)
    BDP = BDP.rename(columns={"volatility_260d":"vola"})
    BDP["vola"] = round(BDP["vola"].astype(float),2)

    for i in range(len(BDP)):
        tmp = con.bdh(BDP.loc[i,"ticker"], ["px_last","tot_return_index_gross_dvds"], "20000101", "20230301")
        
        BDP.loc[i,"PR"] = round(tmp.iloc[len(tmp)-1,0] / tmp.iloc[0,0] - 1,2) 
        BDP.loc[i,"TR"] = round(tmp.iloc[len(tmp)-1,1] / tmp.iloc[0,1] - 1,2) 

        if tmp.iloc[len(tmp)-1,1] - tmp.iloc[len(tmp)-1,0] == 0:
            BDP.loc[i,"PR=TR"] = "1"
        else:
            BDP.loc[i,"PR=TR"] = ""

        BDP.loc[i,"Initial_TD"] = tmp.index[0].strftime("%Y%m%d")
    BDP = BDP.replace("20000104", "-")
    return BDP
# https://www.jpmorgan.com/insights/research/index-research
# https://www.jpmorgan.com/content/dam/jpm/cib/complex/content/markets/index-research/Global-Index-Research-Product-Guide-2022.pdf#page=117

In [8]:
print("Japan Stocks")
L = ["TPX","TPXDDVD"]
BDP(L)

Japan Stocks


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,TPX Index,Tokyo Stock Exchange Tokyo Sto,Tokyo Stock Exchange Tokyo Stock Price Index T...,JPY,15.04,0.16,0.75,,-
1,TPXDDVD Index,Topix Total Return Index JPY,Topix Total Return Index JPY,JPY,15.09,0.75,0.75,1,-


In [9]:
print("Developed Country Stocks")
L = ["MBKO","M0KO","MXKO","M2KO","MAKO","MSHJKOK","GJ124280","HJ124336"]
BDP(L)

Developed Country Stocks


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,MBKO Index,MSCI Kokusai JPY Price Index,MSCI Kokusai JPY Price Index,JPY,20.52,1.77,1.77,1,20000103
1,M0KO Index,MSCI Kokusai Gross Total Retur,MSCI Kokusai Gross Total Return JPY Index,JPY,20.53,3.77,3.77,1,20000103
2,MXKO Index,MSCI KOKUSAI Index,MSCI KOKUSAI Index,USD,18.24,1.06,2.63,,20000103
3,M2KO Index,MSCI Kokusai Gross Total Retur,MSCI Kokusai Gross Total Return USD Index,USD,18.25,2.59,2.59,1,20000103
4,MAKO Index,MSCI Kokusai JPY Net Index,MSCI Kokusai JPY Net Index,JPY,20.53,3.28,3.28,1,20000131
5,MSHJKOK Index,MSCI Hedged Indices In JPY Kok,MSCI Hedged Indices In JPY Kokusai,JPY,NaN,0.44,0.44,1,20000131
6,GJ124280 Index,MSCI Kokusai 100% Hedged to JP,MSCI Kokusai 100% Hedged to JPY Index,JPY,17.32,1.50,1.50,1,20000131
7,HJ124336 Index,MSCI Kokusai 100% Hedged to JP,MSCI Kokusai 100% Hedged to JPY,JPY,17.32,-0.00,-0.00,1,20210226


In [10]:
print("Emerging Markets Stocks")
L = ["GDUEEGF","NDUEEGF","HJ137561"]
BDP(L)

Emerging Markets Stocks


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,GDUEEGF Index,MSCI Daily TR Gross EM USD,MSCI Daily TR Gross EM USD,USD,16.40,2.68,2.68,1,20000103
1,NDUEEGF Index,MSCI Emerging Net Total Return,MSCI Emerging Net Total Return USD Index,USD,16.40,2.41,2.41,1,20000103
2,HJ137561 Index,MSCI EM (Emerging Markets) 100,MSCI EM (Emerging Markets) 100% Hedged to JPY ...,JPY,13.96,1.31,1.31,1,20000131


In [11]:
print("Developed Country REIT")
L = ["SREITWJJ","SREITJWJ"]
BDP(L)

Developed Country REIT


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,SREITWJJ Index,S&P Developed Ex Japan REIT JP,S&P Developed Ex Japan REIT JPY Index,JPY,22.39,2.24,2.24,1,20000103
1,SREITJWJ Index,S&P Developed Ex Japan REIT To,S&P Developed Ex Japan REIT Total Return JPY I...,JPY,22.35,9.12,9.12,1,20000103


In [12]:
print("Japan Bonds")
L = ["BPITTO01"]
BDP(L)

Japan Bonds


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,BPITTO01 Index,BPI Total Index,BPI Total Index,JPY,3.24,0.37,0.37,1,-


In [13]:
print("Developed Country Bond")
L = ["SBWGNJYC","SBWGNJYU"]
BDP(L)

Developed Country Bond


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,SBWGNJYC Index,FTSE Non-JPY WGBI Curr-Hedged,FTSE Non-JPY WGBI Curr-Hedged JPY,JPY,7.32,0.46,0.46,1,20000103
1,SBWGNJYU Index,FTSE Non-JPY WGBI JPY,FTSE Non-JPY WGBI JPY,JPY,11.15,2.15,2.15,1,20000103


In [14]:
print("Emerging Market Bond / USD Bond")
L = ["JPGCUJCP","JPGCHJCP","JPEIDIVR","JPGCCOMP"]
# L = ["JPGCUJCP","JPGCHJCP","JPEIDIVR","JPEIPLUS","JPEMCOMP","GBIE1545"]
tmp = BDP(L)
tmp

Emerging Market Bond / USD Bond


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,JPGCUJCP Index,unhedged Return JPY,J.P. Morgan EMBI Global Diversified unhedged R...,JPY,12.32,1.10,1.10,1,20121001
1,JPGCHJCP Index,Hedged JPY,J.P. Morgan EMBI Global Diversified Hedged JPY,JPY,8.38,1.89,1.89,1,20000103
2,JPEIDIVR Index,JPMorgan Monthly EMBIs,J.P. Morgan EMBI Global Diversified Composite,USD,8.03,3.68,3.68,1,20000103
3,JPGCCOMP Index,Composite,J.P. Morgan EMBI Global Diversified Composite,USD,8.33,3.68,3.68,1,20000103


In [15]:
print("Emerging Market Bond / Local Currency Bond")
L = ["JGENDVUJ","JGENDVHJ","JGENGUJG","JGENGHJG","JGENVUJG","JGENVHJG"]
tmp = BDP(L)
tmp["Remarks"] = ["GBI EM Div"]*2+["GBI EM Global"]*2+["GBI EM Global Div"]*2
tmp

Emerging Market Bond / Local Currency Bond


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD,Remarks
0,JGENDVUJ Index,Divers,J.P. Morgan GBI-EM Diversified JPY Unhedged,JPY,11.81,1.73,1.73,1,20021231,GBI EM Div
1,JGENDVHJ Index,Divers,J.P. Morgan GBI-EM Diversified JPY Hedged,JPY,3.63,0.27,0.27,1,20030101,GBI EM Div
2,JGENGUJG Index,JPY,J.P. Morgan GBI-EM Global Composite Unhedged JPY,JPY,11.62,2.57,2.57,1,20011231,GBI EM Global
3,JGENGHJG Index,JPY,J.P. Morgan GBI-EM Global Composite Hedged JPY,JPY,1.57,0.53,0.53,1,20011231,GBI EM Global
4,JGENVUJG Index,JPY,J.P. Morgan GBI-EM Global Diversified Composit...,JPY,11.08,2.04,2.04,1,20021231,GBI EM Global Div
5,JGENVHJG Index,JPY,J.P. Morgan GBI-EM Global Diversified Composit...,JPY,2.89,0.39,0.39,1,20021231,GBI EM Global Div
